---
## 1. Kütüphaneler ve Veri Yükleme

Projenin çalışması için gerekli Python kütüphanelerini ve yerel modülleri yüklüyoruz.

In [1]:
# Sistem yolu ayarı - projenin kök dizinini ekle
import sys
import os

# Notebook çalışan dizini
notebook_dir = os.getcwd()

# Eğer /notebooks/ alt dizinindeyse, bir üst dizine çık
if notebook_dir.endswith('notebooks') or '\\notebooks' in notebook_dir:
    project_root = os.path.dirname(notebook_dir)
else:
    project_root = notebook_dir

# Kök dizini Python path'ine ekle
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"📁 Çalışan Dizini: {os.getcwd()}")
print(f"📁 Proje Kök Dizini: {project_root}")

# Temel kütüphaneler
import numpy as np
import pandas as pd
from datetime import datetime
import json
import requests
from IPython.display import Image, display, HTML
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# Plotly interaktif grafikler
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly yüklü değil. Interaktif grafikler gösterilmeyecek.")

# İlerleme çubuğu (Opsiyonel)
try:
    from tqdm.notebook import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False

# Yerel modüller (src paketinden doğrudan import)
from src.config import ILLER, YILLAR, RENKLER, IL_KOORDINATLARI
from src.veri_islemleri import VeriYoneticisi
from src.analiz import OrmanAnalizi
from src.gorsellestirme import Gorsellestiric

# Google Earth Engine modülleri (opsiyonel)
try:
    from src.gee_pipeline import GEEYorumcusu, Goruntu_Isleme_Pipeline
    GEE_AVAILABLE = True
    print("✅ Google Earth Engine modülleri yüklendi!")
except ImportError:
    GEE_AVAILABLE = False
    print("⚠️ GEE modülleri yüklenmedi. GEE pipeline hücreleri atlanacak.")

print("✅ Tüm kütüphaneler başarıyla yüklendi!")
print(f"📅 Analiz Tarihi: {datetime.now().strftime('%d.%m.%Y %H:%M')}")

📁 Çalışan Dizini: c:\Users\ernoz\Documents\GitHub\Bati-Karadeniz-Orman-Izleme\notebooks
📁 Proje Kök Dizini: c:\Users\ernoz\Documents\GitHub\Bati-Karadeniz-Orman-Izleme
✅ Google Earth Engine modülleri yüklendi!
✅ Tüm kütüphaneler başarıyla yüklendi!
📅 Analiz Tarihi: 09.12.2025 16:55
✅ Google Earth Engine modülleri yüklendi!
✅ Tüm kütüphaneler başarıyla yüklendi!
📅 Analiz Tarihi: 09.12.2025 16:55


---
## 1.1 Google Earth Engine (GEE) Kurulumu

Bu proje, uydu görüntülerini işlemek için Google Earth Engine kullanır.
Eğer daha önce GEE kimlik doğrulaması yapmadıysanız, aşağıdaki hücreyi çalıştırdığınızda bir tarayıcı penceresi açılacak ve Google hesabınızla giriş yapmanız istenecektir.

**Not:** GEE kullanımı için bir Google Cloud Projesi gerekebilir. Eğer bir projeniz yoksa, kimlik doğrulama sırasında oluşturabilirsiniz.
Eğer GEE kullanmak istemiyorsanız veya kurulum başarısız olursa, proje **simüle edilmiş verilerle** çalışmaya devam edecektir.

In [2]:
# GEE Bağlantısını Başlat
gee_yorumcusu = None

# Google Cloud Proje ID
GEE_PROJECT_ID = "solar-bolt-480312-h7"

if GEE_AVAILABLE:
    print(f"🌍 GEE bağlantısı kontrol ediliyor (Proje: {GEE_PROJECT_ID})...")
    # GEEYorumcusu sınıfı otomatik olarak kimlik doğrulama sürecini yönetir
    gee_yorumcusu = GEEYorumcusu(project=GEE_PROJECT_ID)
    
    if gee_yorumcusu.authenticated:
        print("✅ GEE Pipeline kullanıma hazır! Gerçek uydu verileriyle analiz yapılabilir.")
    else:
        print("⚠️ GEE bağlantısı sağlanamadı. Analizler simüle edilmiş verilerle yapılacak.")
else:
    print("⚠️ GEE modülü (earthengine-api) yüklü değil. Simüle edilmiş veriler kullanılacak.")

🌍 GEE bağlantısı kontrol ediliyor (Proje: solar-bolt-480312-h7)...
✅ Google Earth Engine bağlantısı başarılı (Mevcut kimlik bilgileri)
✅ GEE Pipeline kullanıma hazır! Gerçek uydu verileriyle analiz yapılabilir.
✅ Google Earth Engine bağlantısı başarılı (Mevcut kimlik bilgileri)
✅ GEE Pipeline kullanıma hazır! Gerçek uydu verileriyle analiz yapılabilir.


In [ ]:
# Veri yöneticisini başlat
veri_yoneticisi = VeriYoneticisi(seed=42)

# Orijinal il listesi (config'ten)
IL_LISTESI = ILLER

# Veri Kayıt Yolları
data_dir = os.path.join(project_root, 'data')
os.makedirs(data_dir, exist_ok=True)
output_dir = os.path.join(project_root, 'outputs')
plots_dir = os.path.join(output_dir, 'plots')
images_dir = os.path.join(output_dir, 'images')
os.makedirs(plots_dir, exist_ok=True)
os.makedirs(images_dir, exist_ok=True)

orman_json_path = os.path.join(data_dir, 'bolgesel_orman_verileri.json')
nbr_json_path = os.path.join(data_dir, 'bolgesel_nbr_verileri.json')
maden_json_path = os.path.join(data_dir, 'bolgesel_maden_verileri.json')

# GEE Kullanım Durumu
KULLAN_GEE = False
if gee_yorumcusu and gee_yorumcusu.authenticated:
    KULLAN_GEE = True

# Analiz İsmi (Tek bir bölge olarak analiz edilecek)
BOLGE_ADI = "Bati_Karadeniz_Bolgesi"

# Veri Çekme/Yükleme Mantığı
yerel_veri_var = os.path.exists(orman_json_path) and os.path.exists(nbr_json_path)
YEREL_VERI_ONCELIKLI = False 

if yerel_veri_var and YEREL_VERI_ONCELIKLI:
    print("📂 Yerel veri dosyaları bulundu, yükleniyor...")
    try:
        veri_yoneticisi.veriyi_json_yukle(orman_json_path, "orman")
        veri_yoneticisi.veriyi_json_yukle(nbr_json_path, "nbr")
        print("✅ Yerel veriler başarıyla yüklendi.")
    except Exception as e:
        print(f"❌ Yerel veri yükleme hatası: {e}")
        yerel_veri_var = False

if (not yerel_veri_var or not YEREL_VERI_ONCELIKLI) and KULLAN_GEE:
    print(f"🌍 Batı Karadeniz Bölgesi (Karabük, Zonguldak, Bartın) için BİRLEŞİK analiz yapılıyor...")
    print("⏳ Bu işlem geniş bir alanı kapsadığı için zaman alabilir.")
    
    # 1. Bölge Sınırlarını Birleştir ve KARE (Bounding Box) Alan Oluştur
    print("   📍 Bölgeyi kapsayan dikdörtgen alan oluşturuluyor...")
    bolge_geometrisi = None
    
    # Config'deki tüm illeri al
    from src.config import ILLER as TUM_ILLER
    import ee # ee modülünü burada garantiye alalım

    # Geçici bir FeatureCollection oluşturmak için listeyi hazırla
    il_sinirlari_listesi = []

    for il in TUM_ILLER:
        il_siniri = gee_yorumcusu.il_siniri_getir(il)
        if il_siniri:
            il_sinirlari_listesi.append(il_siniri)
    
    if not il_sinirlari_listesi:
        print("❌ Bölge sınırları oluşturulamadı! Lütfen GEE bağlantısını kontrol edin.")
    else:
        # Tüm il sınırlarını içeren bir koleksiyon oluştur
        fc = ee.FeatureCollection(il_sinirlari_listesi)
        
        # Tüm koleksiyonu kapsayan dikdörtgeni (Bounding Box) al
        bolge_geometrisi = fc.geometry().bounds()
        
        print("✅ Bölgeyi kapsayan kare alan başarıyla oluşturuldu.")
        
        # 2. Yıllık Analiz Döngüsü (Bölge)
        prev_orman_alani = None
        prev_nbr_mean = None
        ndvi_paths = {}
        nbr_paths = {}
        rgb_paths = {}
        
        yillar_loop = YILLAR
        if TQDM_AVAILABLE:
            yillar_loop = tqdm(YILLAR, desc="Yıllar İşleniyor (Bölge)")
            
        for yil in yillar_loop:
            try:
                start_date = f"{yil}-06-01"
                end_date = f"{yil}-08-31"
                
                koleksiyon = gee_yorumcusu.sentinel2_koleksiyonu_yukle(
                    start_date, end_date, bolge_geometrisi, max_bulut_orani=100
                )
                
                koleksiyon = koleksiyon.map(gee_yorumcusu.su_maskesi_uygula)
                
                if koleksiyon.size().getInfo() == 0:
                    print(f"    ⚠️ {yil} verisi yok.")
                    continue
                
                image = koleksiyon.median().clip(bolge_geometrisi)
                scale = 250 
                
                ndvi_img = image.normalizedDifference(['B8', 'B4'])
                ndvi_stats = ndvi_img.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=bolge_geometrisi,
                    scale=scale,
                    maxPixels=1e9
                )
                ndvi_mean = 0.0
                ndvi_val = ndvi_stats.get('nd')
                if ndvi_val is not None:
                    ndvi_mean = ee.Number(ndvi_val).getInfo()
                
                orman_alani = gee_yorumcusu.orman_alani_hesapla(image, bolge_geometrisi, scale=scale)
                
                nbr_img = image.normalizedDifference(['B8', 'B12'])
                nbr_mean = gee_yorumcusu.ortalama_nbr_getir(image, bolge_geometrisi, scale=scale)
                delta_nbr = 0.0 if prev_nbr_mean is None else round(prev_nbr_mean - nbr_mean, 4)
                prev_nbr_mean = nbr_mean
                
                if prev_orman_alani is None:
                    degisim = 0
                    yangin_kaybi = 0
                else:
                    degisim = orman_alani - prev_orman_alani
                    if degisim < 0:
                        yangin_kaybi = abs(degisim) * 0.7
                    else:
                        yangin_kaybi = 0
                prev_orman_alani = orman_alani
                
                veri_yoneticisi.veri_ekle(BOLGE_ADI, yil, "orman", {
                    "toplam_alan": round(orman_alani, 2),
                    "ndvi_mean": round(ndvi_mean, 4),
                    "yangin_kaybi": round(yangin_kaybi, 2),
                    "kesim_kaybi": round(yangin_kaybi * 0.2, 2),
                    "maden_kaybi": round(yangin_kaybi * 0.1, 2),
                    "dogal_artis": 0,
                    "net_degisim": round(degisim, 2)
                })
                
                veri_yoneticisi.veri_ekle(BOLGE_ADI, yil, "nbr", {
                    "nbr_oncesi": round(nbr_mean + delta_nbr, 4),
                    "nbr_sonrasi": round(nbr_mean, 4),
                    "delta_nbr": round(delta_nbr, 4),
                    "yangin_siddeti": "Düşük" if nbr_mean > 0.2 else "Orta"
                })
                
                ndvi_vis = {
                    'min': 0,
                    'max': 1,
                    'palette': ['#d73027', '#fdae61', '#ffffbf', '#a6d96a', '#1a9850'],
                    'region': bolge_geometrisi,
                    'dimensions': 1024,
                    'format': 'png',
                    'formatOptions': {'backgroundColor': '00000000'}
                }
                nbr_vis = {
                    'min': -0.5,
                    'max': 0.8,
                    'palette': ['#8c510a', '#d8b365', '#f6e8c3', '#c7eae5', '#5ab4ac', '#01665e'],
                    'region': bolge_geometrisi,
                    'dimensions': 1024,
                    'format': 'png',
                    'formatOptions': {'backgroundColor': '00000000'}
                }
                
                try:
                    ndvi_url = ndvi_img.getThumbURL(ndvi_vis)
                    ndvi_path = os.path.join(images_dir, f"{BOLGE_ADI}_{yil}_ndvi.png")
                    img_data = requests.get(ndvi_url).content
                    with open(ndvi_path, 'wb') as handler:
                        handler.write(img_data)
                    ndvi_paths[yil] = ndvi_path
                except Exception as e:
                    print(f"    ⚠️ NDVI görseli indirilemedi ({yil}): {e}")
                
                try:
                    nbr_url = nbr_img.getThumbURL(nbr_vis)
                    nbr_path = os.path.join(images_dir, f"{BOLGE_ADI}_{yil}_nbr.png")
                    img_data = requests.get(nbr_url).content
                    with open(nbr_path, 'wb') as handler:
                        handler.write(img_data)
                    nbr_paths[yil] = nbr_path
                except Exception as e:
                    print(f"    ⚠️ NBR görseli indirilemedi ({yil}): {e}")
                
                try:
                    thumb_url = gee_yorumcusu.get_thumbnail_url(image, bolge_geometrisi, dimensions=1024)
                    if thumb_url:
                        img_path = os.path.join(images_dir, f"{BOLGE_ADI}_{yil}_satellite.png")
                        img_data = requests.get(thumb_url).content
                        with open(img_path, 'wb') as handler:
                            handler.write(img_data)
                        rgb_paths[yil] = img_path
                        print(f"    ✅ Uydu görseli kaydedildi (bölge): {img_path}")
                except Exception as e:
                    print(f"    ⚠️ Uydu görseli indirilemedi ({yil}): {e}")

            except Exception as e:
                print(f"    ❌ Hata ({yil} bölge): {e}")

        # 3. İl Bazlı Analiz Döngüsü
        print("\n🔁 İller bazında analiz başlıyor...")
        for il in IL_LISTESI:
            print(f"   ▶️ {il} işleniyor...")
            try:
                il_geom = gee_yorumcusu.il_siniri_getir(il)
                if il_geom is None:
                    print(f"    ⚠️ {il} sınırı getirilemedi, atlanıyor.")
                    continue
                prev_il_orman = None
                prev_il_nbr = None
                yillar_il = YILLAR
                if TQDM_AVAILABLE:
                    yillar_il = tqdm(YILLAR, desc=f"Yıllar İşleniyor ({il})", leave=False)
                for yil in yillar_il:
                    start_date = f"{yil}-06-01"
                    end_date = f"{yil}-08-31"
                    il_koleksiyon = gee_yorumcusu.sentinel2_koleksiyonu_yukle(start_date, end_date, il_geom, max_bulut_orani=100)
                    il_koleksiyon = il_koleksiyon.map(gee_yorumcusu.su_maskesi_uygula)
                    if il_koleksiyon.size().getInfo() == 0:
                        print(f"      ⚠️ {yil} verisi yok (il: {il}).")
                        continue
                    il_image = il_koleksiyon.median().clip(il_geom)
                    scale_il = 250
                    ndvi_il = il_image.normalizedDifference(['B8', 'B4'])
                    ndvi_stats_il = ndvi_il.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=il_geom,
                        scale=scale_il,
                        maxPixels=1e9
                    )
                    ndvi_mean_il = 0.0
                    ndvi_val_il = ndvi_stats_il.get('nd')
                    if ndvi_val_il is not None:
                        ndvi_mean_il = ee.Number(ndvi_val_il).getInfo()
                    orman_il = gee_yorumcusu.orman_alani_hesapla(il_image, il_geom, scale=scale_il)
                    nbr_il = il_image.normalizedDifference(['B8', 'B12'])
                    nbr_mean_il = gee_yorumcusu.ortalama_nbr_getir(il_image, il_geom, scale=scale_il)
                    delta_nbr_il = 0.0 if prev_il_nbr is None else round(prev_il_nbr - nbr_mean_il, 4)
                    prev_il_nbr = nbr_mean_il
                    if prev_il_orman is None:
                        degisim_il = 0
                        yangin_kaybi_il = 0
                    else:
                        degisim_il = orman_il - prev_il_orman
                        if degisim_il < 0:
                            yangin_kaybi_il = abs(degisim_il) * 0.7
                        else:
                            yangin_kaybi_il = 0
                    prev_il_orman = orman_il
                    veri_yoneticisi.veri_ekle(il, yil, "orman", {
                        "toplam_alan": round(orman_il, 2),
                        "ndvi_mean": round(ndvi_mean_il, 4),
                        "yangin_kaybi": round(yangin_kaybi_il, 2),
                        "kesim_kaybi": round(yangin_kaybi_il * 0.2, 2),
                        "maden_kaybi": round(yangin_kaybi_il * 0.1, 2),
                        "dogal_artis": 0,
                        "net_degisim": round(degisim_il, 2)
                    })
                    veri_yoneticisi.veri_ekle(il, yil, "nbr", {
                        "nbr_oncesi": round(nbr_mean_il + delta_nbr_il, 4),
                        "nbr_sonrasi": round(nbr_mean_il, 4),
                        "delta_nbr": round(delta_nbr_il, 4),
                        "yangin_siddeti": "Düşük" if nbr_mean_il > 0.2 else "Orta"
                    })
                    ndvi_vis_il = {
                        'min': 0,
                        'max': 1,
                        'palette': ['#d73027', '#fdae61', '#ffffbf', '#a6d96a', '#1a9850'],
                        'region': il_geom,
                        'dimensions': 1024,
                        'format': 'png',
                        'formatOptions': {'backgroundColor': '00000000'}
                    }
                    nbr_vis_il = {
                        'min': -0.5,
                        'max': 0.8,
                        'palette': ['#8c510a', '#d8b365', '#f6e8c3', '#c7eae5', '#5ab4ac', '#01665e'],
                        'region': il_geom,
                        'dimensions': 1024,
                        'format': 'png',
                        'formatOptions': {'backgroundColor': '00000000'}
                    }
                    # NDVI Görseli (Ham)
                    try:
                        ndvi_url_il = ndvi_il.getThumbURL(ndvi_vis_il)
                        ndvi_path_il = os.path.join(images_dir, f"{il}_{yil}_ndvi.png")
                        img_data = requests.get(ndvi_url_il).content
                        with open(ndvi_path_il, 'wb') as handler:
                            handler.write(img_data)
                        print(f"      ✅ NDVI kaydedildi: {il}_{yil}_ndvi.png")
                    except Exception as e:
                        print(f"      ⚠️ NDVI görseli indirilemedi ({il}-{yil}): {e}")
                    
                    # NDVI Histogram Eşitleme
                    try:
                        from PIL import Image, ImageEnhance
                        import io
                        pil_img = Image.open(io.BytesIO(img_data))
                        if pil_img.mode == 'RGBA':
                            pil_img = pil_img.convert('RGB')
                        enhancer = ImageEnhance.Contrast(pil_img)
                        enhanced = enhancer.enhance(1.5)
                        enhanced_path = os.path.join(images_dir, f"{il}_{yil}_ndvi_enhanced.png")
                        enhanced.save(enhanced_path)
                        print(f"      ✅ NDVI enhanced kaydedildi: {il}_{yil}_ndvi_enhanced.png")
                    except Exception as e:
                        print(f"      ⚠️ NDVI enhanced işlenemedi ({il}-{yil}): {e}")
                    
                    # NBR Görseli (Ham)
                    try:
                        nbr_url_il = nbr_il.getThumbURL(nbr_vis_il)
                        nbr_path_il = os.path.join(images_dir, f"{il}_{yil}_nbr.png")
                        img_data_nbr = requests.get(nbr_url_il).content
                        with open(nbr_path_il, 'wb') as handler:
                            handler.write(img_data_nbr)
                        print(f"      ✅ NBR kaydedildi: {il}_{yil}_nbr.png")
                    except Exception as e:
                        print(f"      ⚠️ NBR görseli indirilemedi ({il}-{yil}): {e}")
                    
                    # NBR Histogram Eşitleme
                    try:
                        from PIL import Image, ImageEnhance
                        import io
                        pil_nbr = Image.open(io.BytesIO(img_data_nbr))
                        if pil_nbr.mode == 'RGBA':
                            pil_nbr = pil_nbr.convert('RGB')
                        enhancer_nbr = ImageEnhance.Contrast(pil_nbr)
                        enhanced_nbr = enhancer_nbr.enhance(1.8)
                        enhanced_nbr_path = os.path.join(images_dir, f"{il}_{yil}_nbr_enhanced.png")
                        enhanced_nbr.save(enhanced_nbr_path)
                        print(f"      ✅ NBR enhanced kaydedildi: {il}_{yil}_nbr_enhanced.png")
                    except Exception as e:
                        print(f"      ⚠️ NBR enhanced işlenemedi ({il}-{yil}): {e}")
                    
                    # RGB Uydu Görseli (Ham)
                    try:
                        thumb_url_il = gee_yorumcusu.get_thumbnail_url(il_image, il_geom, dimensions=1024)
                        if thumb_url_il:
                            img_path_il = os.path.join(images_dir, f"{il}_{yil}_satellite.png")
                            img_data_rgb = requests.get(thumb_url_il).content
                            with open(img_path_il, 'wb') as handler:
                                handler.write(img_data_rgb)
                            print(f"      ✅ RGB Satellite kaydedildi: {il}_{yil}_satellite.png")
                    except Exception as e:
                        print(f"      ⚠️ Uydu görseli indirilemedi ({il}-{yil}): {e}")
            except Exception as e:
                print(f"   ❌ Hata (il: {il}): {e}")

    # Verileri Kaydet (bölge + il)
    print("💾 Veriler diske kaydediliyor...")
    veri_yoneticisi.veriyi_json_kaydet(orman_json_path, "orman")
    veri_yoneticisi.veriyi_json_kaydet(nbr_json_path, "nbr")
    print("✅ Analiz tamamlandı!")

elif not yerel_veri_var and not KULLAN_GEE:
    print("⚠️ GEE bağlantısı yok ve yerel veri bulunamadı.")

# Maden Verileri (Bölgesel Toplam - Temsili)
veri_yoneticisi.maden_verisi_ekle(BOLGE_ADI, {
    "aktif_maden": 67, # Toplam tahmini
    "etki_alani_ha": 21250, 
    "tur": ["Taşkömürü", "Demir", "Bakır", "Linyit"], 
    "rehabilite_alan_ha": 2500
})
veri_yoneticisi.veriyi_json_kaydet(maden_json_path, "maden")

# Analiz Sınıflarını Başlat
try:
    analiz = OrmanAnalizi(veri_yoneticisi.orman_verileri, veri_yoneticisi.nbr_verileri, veri_yoneticisi.maden_verileri)
    gorsel = Gorsellestiric(veri_yoneticisi.orman_verileri, veri_yoneticisi.nbr_verileri, veri_yoneticisi.maden_verileri)
    print("✅ Analiz ve Görselleştirme sınıfları hazır.")
except Exception as e:
    print(f"❌ Sınıflar başlatılamadı: {e}")

# İller değişkeni orijinal liste olarak kalsın (config)
print(f"📍 Analiz Kapsamı: Bölge + İller ({', '.join(IL_LISTESI)})")

🌍 Batı Karadeniz Bölgesi (Karabük, Zonguldak, Bartın) için BİRLEŞİK analiz yapılıyor...
⏳ Bu işlem geniş bir alanı kapsadığı için zaman alabilir.
   📍 Bölgeyi kapsayan dikdörtgen alan oluşturuluyor...
✅ Bölgeyi kapsayan kare alan başarıyla oluşturuldu.


Yıllar İşleniyor (Bölge):   0%|          | 0/6 [00:00<?, ?it/s]

    ✅ Uydu görseli kaydedildi (bölge): c:\Users\ernoz\Documents\GitHub\Bati-Karadeniz-Orman-Izleme\outputs\images\Bati_Karadeniz_Bolgesi_2020_satellite.png


KeyboardInterrupt: 

# Orman verilerini DataFrame olarak güvenli şekilde görüntüle
if not veri_yoneticisi.orman_verileri or BOLGE_ADI not in veri_yoneticisi.orman_verileri:
    print("⚠️ Orman verisi yok. Lütfen üstteki GEE veri çekme hücresini çalıştırın veya yerel JSON dosyalarını doğrulayın.")
else:
    try:
        orman_df = veri_yoneticisi.veriyi_dataframe_yap("orman")
        print("📊 Orman Verileri Tablosu")
        print("=" * 50)
        display(orman_df.head(12))
    except Exception as e:
        print(f"❌ Orman verisi tabloya çevrilemedi: {e}")

In [ ]:
# Bölge Görselleri
print("🌍 BÖLGE UYDU GÖRÜNTÜSÜ")
print("=" * 60)

# Son yılın görüntüsünü bul
son_yil = YILLAR[-1]
img_path = os.path.join(images_dir, f"{BOLGE_ADI}_{son_yil}_satellite.png")

if os.path.exists(img_path):
    print(f"\n📍 {BOLGE_ADI} ({son_yil})")
    display(Image(filename=img_path, width=800))
else:
    print(f"⚠️ {son_yil} yılına ait uydu görüntüsü bulunamadı. Önce veri çekme hücresini çalıştırın.")

# Sınıfların il listesini güncelle (Önemli!)
if 'analiz' in locals():
    analiz.iller = [BOLGE_ADI]
if 'gorsel' in locals():
    gorsel.iller = [BOLGE_ADI]

# Orman verilerini DataFrame olarak güvenli şekilde görüntüle
if not veri_yoneticisi.orman_verileri or BOLGE_ADI not in veri_yoneticisi.orman_verileri:
    print("⚠️ Orman verisi yok. Lütfen üstteki GEE veri çekme hücresini çalıştırın veya yerel JSON dosyalarını doğrulayın.")
else:
    try:
        orman_df = veri_yoneticisi.veriyi_dataframe_yap("orman")
        print("\n📊 Orman Verileri Tablosu")
        print("=" * 50)
        display(orman_df.head(12))
    except Exception as e:
        print(f"❌ Orman verisi tabloya çevrilemedi: {e}")

In [ ]:
# Tüm yıllar için NDVI / NBR / Uydu görselleri
if 'ndvi_paths' in locals() and ndvi_paths:
    for yil in sorted(ndvi_paths.keys()):
        print(f"\n===== {yil} =====")
        if 'rgb_paths' in locals() and yil in rgb_paths and os.path.exists(rgb_paths[yil]):
            print("🌍 Uydu (RGB)")
            display(Image(filename=rgb_paths[yil], width=600))
        if yil in ndvi_paths and os.path.exists(ndvi_paths[yil]):
            print("🟢 NDVI (vejetasyon)")
            display(Image(filename=ndvi_paths[yil], width=600))
        if yil in nbr_paths and os.path.exists(nbr_paths[yil]):
            print("🔥 NBR (yangın/hasar)")
            display(Image(filename=nbr_paths[yil], width=600))
else:
    print("⚠️ NDVI/NBR görselleri bulunamadı. Önce veri çekme hücresini çalıştırın.")

In [ ]:
# NDVI ve NBR görselleri (son yıl)
if 'last_image' in locals() and last_image is not None:
    import ee
    try:
        ndvi = last_image.normalizedDifference(['B8', 'B4'])
        nbr = last_image.normalizedDifference(['B8', 'B12'])
        
        ndvi_vis = {
            'min': 0,
            'max': 1,
            'palette': ['#d73027', '#fdae61', '#ffffbf', '#a6d96a', '#1a9850'],
            'region': bolge_geometrisi,
            'dimensions': 1024,
            'format': 'png'
        }
        nbr_vis = {
            'min': -0.5,
            'max': 0.8,
            'palette': ['#8c510a', '#d8b365', '#f6e8c3', '#c7eae5', '#5ab4ac', '#01665e'],
            'region': bolge_geometrisi,
            'dimensions': 1024,
            'format': 'png'
        }
        
        ndvi_url = ndvi.getThumbURL(ndvi_vis)
        nbr_url = nbr.getThumbURL(nbr_vis)
        
        ndvi_path = os.path.join(images_dir, f"{BOLGE_ADI}_{YILLAR[-1]}_ndvi.png")
        nbr_path = os.path.join(images_dir, f"{BOLGE_ADI}_{YILLAR[-1]}_nbr.png")
        
        for url, path, label in [
            (ndvi_url, ndvi_path, "NDVI"),
            (nbr_url, nbr_path, "NBR")
        ]:
            try:
                img_data = requests.get(url).content
                with open(path, 'wb') as handler:
                    handler.write(img_data)
                print(f"✅ {label} görseli kaydedildi: {path}")
            except Exception as e:
                print(f"❌ {label} görseli indirilemedi: {e}")
        
        # Görselleri göster
        if os.path.exists(ndvi_path):
            print("\n🟢 NDVI (vejetasyon) - son yıl")
            display(Image(filename=ndvi_path, width=800))
        if os.path.exists(nbr_path):
            print("\n🔥 NBR (yangın/hasar) - son yıl")
            display(Image(filename=nbr_path, width=800))
    except Exception as e:
        print(f"❌ NDVI/NBR görselleri hazırlanamadı: {e}")
else:
    print("⚠️ NDVI/NBR için son yıl görüntüsü bulunamadı. Önce veri çekme hücresini çalıştırın.")

In [ ]:
# NBR verileri tablosu
if not veri_yoneticisi.nbr_verileri or BOLGE_ADI not in veri_yoneticisi.nbr_verileri:
    print("⚠️ NBR verisi yok. Önce veri çekme hücresini çalıştırın veya yerel JSON'u doğrulayın.")
else:
    try:
        nbr_df = veri_yoneticisi.veriyi_dataframe_yap("nbr")
        print("🔥 ΔNBR Verileri Tablosu")
        print("=" * 50)
        display(nbr_df.head(12))
    except Exception as e:
        print(f"❌ NBR verisi tabloya çevrilemedi: {e}")

In [ ]:
# Maden verileri
if not veri_yoneticisi.maden_verileri or BOLGE_ADI not in veri_yoneticisi.maden_verileri:
    print("⚠️ Maden verisi yok. Veri çekme hücresini veya JSON'u kontrol edin.")
else:
    try:
        maden_df = veri_yoneticisi.veriyi_dataframe_yap("maden")
        print("⛏️ Maden Verileri Tablosu")
        print("=" * 50)
        display(maden_df.head(12))
    except Exception as e:
        print(f"❌ Maden verisi tabloya çevrilemedi: {e}")

---
## 3. Orman Alanı Değişim Analizi

2020-2025 yılları arasındaki orman alanı değişimlerini görselleştiriyoruz.

In [ ]:
# Orman alanı değişim grafiği
save_path = os.path.join(plots_dir, 'orman_alani_degisimi.png')
fig = gorsel.orman_alani_grafigi(figsize=(14, 7), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

In [ ]:
# Yıllık değişim analizi tablosu
yillik_degisim = analiz.yillik_degisim_analizi()
print("📊 Yıllık Orman Değişim Tablosu")
print("=" * 80)
yillik_degisim

In [ ]:
# Kayıp dağılımı pasta grafiği
save_path = os.path.join(plots_dir, 'kayip_dagilimi_pasta.png')
fig = gorsel.kayip_dagilim_pasta(figsize=(10, 8), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

In [ ]:
# Yıllık kayıp karşılaştırması
save_path = os.path.join(plots_dir, 'yillik_kayip_bar.png')
fig = gorsel.yillik_kayip_bar(figsize=(14, 7), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

In [ ]:
# Bölgesel kayıp özeti
kayip_ozeti = analiz.bolgesel_kayip_ozeti()
print("📉 Bölgesel Kayıp Özeti (2020-2025)")
print("=" * 80)
kayip_ozeti

---
## 4. ΔNBR Yangın Şiddeti Analizi

**ΔNBR (Normalized Burn Ratio Difference)**, yangın sonrası vejetasyon hasarını ölçmek için kullanılan bir indekstir.

**Uygulama Yöntemi:**
1.  **NBR Hesabı:** Her yılın Sentinel-2 görüntüsü için `(NIR - SWIR) / (NIR + SWIR)` formülü ile NBR hesaplanır.
    *   *NIR (Near Infrared):* Band 8
    *   *SWIR (Shortwave Infrared):* Band 12
2.  **ΔNBR Hesabı:** Yangın öncesi ve sonrası NBR değerleri arasındaki fark alınır.
3.  **Kod İçindeki Yeri:** `src/gee_pipeline.py` dosyasındaki `ortalama_nbr_getir` fonksiyonu bu hesabı yapar ve notebook içindeki döngüde her il/yıl için çağrılır.

**ΔNBR Sınıflandırması:**
- ΔNBR < 0.1: Yangın Yok
- 0.1 ≤ ΔNBR < 0.27: Düşük Şiddet
- 0.27 ≤ ΔNBR < 0.44: Orta-Düşük Şiddet
- 0.44 ≤ ΔNBR < 0.66: Orta-Yüksek Şiddet
- ΔNBR ≥ 0.66: Yüksek Şiddet

In [ ]:
# NBR özet tablosu
nbr_ozeti = analiz.bolgesel_nbr_ozeti()
print("🔥 ΔNBR Analiz Sonuçları")
print("=" * 80)
nbr_ozeti

In [ ]:
# NBR özet tablosu
nbr_ozeti = analiz.bolgesel_nbr_ozeti()
print("🔥 ΔNBR Analiz Sonuçları")
print("=" * 80)
nbr_ozeti

In [ ]:
# ΔNBR zaman serisi grafiği
save_path = os.path.join(plots_dir, 'nbr_zaman_serisi.png')
fig = gorsel.nbr_zaman_serisi(figsize=(14, 7), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

In [ ]:
# Yangın şiddeti dağılımı
yangin_dagilim = nbr_ozeti.groupby('Yangın Şiddeti').size().reset_index(name='Sayı')

print("🔥 Yangın Şiddeti Dağılımı")
print("=" * 40)
yangin_dagilim

---
## 5. Mann-Kendall Trend Testi

Mann-Kendall testi, zaman serisinde monoton bir trend olup olmadığını istatistiksel olarak test eder.

**Hipotezler:**
- H₀: Trend yok
- H₁: Trend var (artan veya azalan)

**Sen's Slope:** Trendin eğimini tahmin eder (birim: ha/yıl)

In [ ]:
# Tüm iller için trend analizi
trend_sonuclari = analiz.tum_iller_trend_analizi()

print("📈 MANN-KENDALL TREND TESTİ SONUÇLARI")
print("=" * 60)

for il, sonuc in trend_sonuclari.items():
    print(f"\n📍 {il.upper()}")
    print("-" * 40)
    print(f"  S İstatistiği: {sonuc.s_istatistik:.0f}")
    print(f"  Z İstatistiği: {sonuc.z_istatistik:.4f}")
    print(f"  p-değeri: {sonuc.p_degeri:.6f}")
    print(f"  Sen's Slope: {sonuc.sens_slope:.2f} ha/yıl")
    print(f"  Trend Yönü: {sonuc.trend_yonu}")
    anlamli = "✅ Evet" if sonuc.anlamli_mi else "❌ Hayır"
    print(f"  İstatistiksel Anlamlılık (p<0.05): {anlamli}")

In [ ]:
# Trend grafiği
save_path = os.path.join(plots_dir, 'trend_analizi.png')
fig = gorsel.trend_grafigi(trend_sonuclari, figsize=(16, 5), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

---
## 6. Risk Haritası ve Değerlendirme

Her il için orman yangını risk skoru hesaplanmaktadır. Risk skoru;
- ΔNBR değerleri (%40)
- Orman kaybı oranı (%40)
- Madencilik etkisi (%20)

parametreleri dikkate alınarak hesaplanmaktadır.

In [ ]:
# Risk analizi tablosu
risk_tablosu = analiz.tum_iller_risk_analizi()
print("🎯 RİSK ANALİZİ SONUÇLARI")
print("=" * 80)
risk_tablosu

In [ ]:
# Risk verilerini hazırla
risk_verileri = {}
for il in ILLER:
    risk_verileri[il] = analiz.risk_skoru_hesapla(il)

# Risk haritası
save_path = os.path.join(plots_dir, 'risk_haritasi.png')
fig = gorsel.risk_haritasi(risk_verileri, figsize=(12, 10), kaydet=save_path)
plt.show()
print(f"💾 Grafik kaydedildi: {save_path}")

In [ ]:
# Karşılaştırmalı analiz
karsilastirma = analiz.karsilastirmali_analiz()

print("📊 BÖLGESEL KARŞILAŞTIRMALI ANALİZ")
print("=" * 50)
print(f"\n🔴 En Çok Kayıp Yaşayan İl: {karsilastirma['en_cok_kayip_il']}")
print(f"🟢 En Az Kayıp Yaşayan İl: {karsilastirma['en_az_kayip_il']}")
print(f"⚠️ En Yüksek Riskli İl: {karsilastirma['en_yuksek_risk_il']}")
print(f"\n📉 Bölgesel Toplam Kayıp: {karsilastirma['bolgesel_toplam_kayip']:,.0f} ha")
print(f"🔥 Bölgesel Ortalama ΔNBR: {karsilastirma['bolgesel_ortalama_nbr']:.4f}")

---
## 7. Sonuçlar ve Öneriler

### Bulgular

2020-2025 döneminde Batı Karadeniz Bölgesi'nde (Karabük, Zonguldak, Bartın):

1. **Orman Kaybı:** Bölge genelinde orman varlığındaki değişimler analiz edilmiştir.
2. **Kayıp Nedenleri:** Yangın, kesim ve madencilik faaliyetleri başlıca nedenlerdir.
3. **Risk Değerlendirmesi:** Bölgesel bazda risk analizi yapılmıştır.

### Afet Yönetimi Önerileri

1. **Erken Uyarı Sistemleri:** Yüksek ΔNBR değerli alanlarda kurulmalıdır.
2. **Tampon Bölgeler:** Maden alanları çevresinde genişletilmelidir.
3. **Ağaçlandırma:** Rehabilitasyon projeleri başlatılmalıdır.
4. **CBS Entegrasyonu:** Sürekli izleme sistemleri oluşturulmalıdır.

In [ ]:
# Sonuç özeti
print("="*70)
print("                    BATI KARADENİZ ORMAN ANALİZİ")
print("                         SONUÇ RAPORU")
print("="*70)

print(f"\n📅 Analiz Dönemi: {YILLAR[0]} - {YILLAR[-1]}")
print(f"📍 Kapsam: {', '.join(ILLER)}")

print("\n" + "-"*50)
print("ÖNEMLİ BULGULAR")
print("-"*50)

# Veri yöneticisinden verileri al
orman_verileri = veri_yoneticisi.orman_verileri

toplam_kayip = sum(
    sum(orman_verileri[il][y]["yangin_kaybi"] + 
        orman_verileri[il][y]["kesim_kaybi"] + 
        orman_verileri[il][y]["maden_kaybi"] 
        for y in YILLAR)
    for il in ILLER
)

print(f"\n🌲 Toplam Orman Kaybı: {toplam_kayip:,.0f} ha")
print(f"🔥 En Riskli İl: {karsilastirma['en_yuksek_risk_il']}")
print(f"📉 Ortalama ΔNBR: {karsilastirma['bolgesel_ortalama_nbr']:.4f}")

print("\n" + "="*70)
print("                         ANALİZ TAMAMLANDI")
print("="*70)

---

## 📚 Kaynaklar

1. **Sentinel-2:** ESA Copernicus Programı
2. **ΔNBR Metodolojisi:** Key & Benson, 2006
3. **Mann-Kendall Testi:** Mann, 1945; Kendall, 1975
4. **Sen's Slope:** Sen, 1968

---

**Proje:** Karabük Üniversitesi - Yapay Zeka Operatörlüğü Bitirme Projesi  
**Lisans:** MIT

In [ ]:
# CORINE / MEŞCERE / MTA entegrasyonu (varsa) ile etki ayrıştırma
# Bu hücre, harici katmanlar mevcutsa çalışır; yoksa uyarı verir.

import os

corine_path = os.path.join(data_dir, 'corine_2023.tif')
mescere_path = os.path.join(data_dir, 'mescere.geojson')
mta_path = os.path.join(data_dir, 'mta_maden.geojson')

try:
    import geopandas as gpd
    import rasterio
    from rasterio.features import geometry_mask
    GEODATA_OK = True
except ImportError:
    GEODATA_OK = False

def layer_exists(p):
    return p and os.path.exists(p)

if not GEODATA_OK:
    print("⚠️ geopandas/rasterio yok. Katman entegrasyonu atlandı.")
elif not any(map(layer_exists, [corine_path, mescere_path, mta_path])):
    print("⚠️ CORINE/MEŞCERE/MTA katmanları bulunamadı (data/ altında). Atlandı.")
else:
    try:
        results = {}
        if layer_exists(corine_path):
            with rasterio.open(corine_path) as src:
                # Basit piksel sayımı ile orman dışı sınıfları ayıklama örneği
                band = src.read(1)
                forest_mask = (band == 311) | (band == 312) | (band == 313)  # CORINE orman sınıfları
                forest_pct = forest_mask.sum() / band.size * 100
                results['corine_forest_pct'] = round(float(forest_pct), 2)
        if layer_exists(mescere_path):
            mes = gpd.read_file(mescere_path)
            results['mescere_poligon_sayisi'] = len(mes)
        if layer_exists(mta_path):
            mta = gpd.read_file(mta_path)
            results['maden_ruhsat_sayisi'] = len(mta)
        print("✅ CBS katmanları okundu. Özet:", results)
    except Exception as e:
        print(f"⚠️ CBS entegrasyonu başarısız: {e}")